# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### Импорт бибилиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
RND = np.random.RandomState(12345)

### Загрузка данных

In [2]:
geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

### Проверка данных

Проверка на пропуски

In [3]:
print(geo_0.info(), geo_1.info(), geo_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Проверка на дубликаты

In [4]:
print(geo_0.duplicated().sum())
print(geo_1.duplicated().sum())
print(geo_2.duplicated().sum())

0
0
0


In [5]:
print(geo_0.describe())
print(geo_0.corr())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


In [23]:
print(geo_1.describe())
print(geo_1.corr())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.734063       5.019721     137.945408
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


In [24]:
print(geo_2.describe())
print(geo_2.corr())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.844801      16.739402     190.029838
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


### Вывод

- Тип данных столбцов корректный
- Пропуски отсутствуют
- Дубликаты отсутствуют

`Регион № 1` имеется зависимость между показателем 'F2' и 'Product' (0.48), занимает второе место по среднему запасу 'Product'

`Регион № 2` имеется очень сильная зависимость между показателем 'F2' и 'Product' (1.00), последнее место по среднему запасу 'Product'

`Регион № 3` имеется зависимость между показателем 'F2' и 'Product' (0.45), занимает первое место по среднему запасу 'Product'

## Обучение и проверка модели

### Разбейте данные на обучающую и валидационную выборки

In [8]:
features_0 = geo_0.drop(['id', 'product'], axis = 1)
target_0 = geo_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=.25, random_state=RND)

features_1 = geo_1.drop(['id', 'product'], axis = 1)
target_1 = geo_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=.25, random_state=RND)

features_2 = geo_2.drop(['id', 'product'], axis = 1)
target_2 = geo_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=.25, random_state=RND)

### Обучите модель, cохраните средний запас предсказанного сырья и RMSE

In [9]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = model_0.predict(features_valid_0)
MSE_0 = mean_squared_error(target_valid_0, predictions_0)
RMSE_0 = MSE_0 ** 0.5

print('Cредний запас предсказанного сырья', predictions_0.mean())
print('RMSE', RMSE_0)

Cредний запас предсказанного сырья 92.59256778438035
RMSE 37.5794217150813


In [10]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)
MSE_1 = mean_squared_error(target_valid_1, predictions_1)
RMSE_1 = MSE_1 ** 0.5

print('Cредний запас предсказанного сырья', predictions_1.mean())
print('RMSE', RMSE_1)

Cредний запас предсказанного сырья 68.76995145799754
RMSE 0.889736773768065


In [11]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = model_2.predict(features_valid_2)
MSE_2 = mean_squared_error(target_valid_2, predictions_2)
RMSE_2 = MSE_2 ** 0.5

print('Cредний запас предсказанного сырья',predictions_2.mean())
print('RMSE',RMSE_2)

Cредний запас предсказанного сырья 95.087528122523
RMSE 39.958042459521614


### Вывод

- Наилучшая RMSE в первом регионе
- Наибольшая средняя по запасам в третьем регионе 

## Подготовка к расчёту прибыли

### Достаточный объём сырья для безубыточной разработки новой скважины

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [25]:
well = 500
wells_number = 200
budget = 10**10
per_unit = 450000

In [13]:
budget_for_one = budget / wells_number
min_mean_product = budget_for_one / per_unit
print('Объём сырья для безубыточной разработки новой скважины:', min_mean_product)
print('\nCредний запас сырья регион 1:',target_0.mean(),
'\nCредний запас сырья регион 2:', target_1.mean(),
'\nCредний запас сырья регион 3:', target_2.mean())

Объём сырья для безубыточной разработки новой скважины: 111.11111111111111

Cредний запас сырья регион 1: 92.50000000000001 
Cредний запас сырья регион 2: 68.82500000000002 
Cредний запас сырья регион 3: 95.00000000000004


Средний запас сырья в каждом из трех регионов меньше расчетного объёма сырья для безубыточной разработки новой скважины

### Напишите функцию для расчёта прибыли по выбранным 200 скважинам и предсказаниям модели

In [14]:
def revenue(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:wells_number]
    return (round((selected.sum()*per_unit) - budget,2))

In [15]:
print('Прибыль регион 1:',revenue(target_valid_0,predictions_0))
print('Прибыль регион 2:',revenue(target_valid_1,predictions_1))
print('Прибыль регион 3:',revenue(target_valid_2,predictions_2))

Прибыль регион 1: 3320826043.14
Прибыль регион 2: 2415086696.68
Прибыль регион 3: 2539915945.84


### Вывод

Самым прибыльным регионом является `Регион 1`, расчетный показатель прибыли составляет 3 320 826 043,14 рублей

## Расчёт прибыли и рисков 

### Распределение прибыли техникой Bootstrap с 1000 выборок

In [16]:
target_valid_0.reset_index(drop= True , inplace= True )
target_valid_1.reset_index(drop= True , inplace= True )
target_valid_2.reset_index(drop= True , inplace= True )

In [17]:
def Bootstrap(target, probabilities):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=well, replace=True, random_state=RND)
        probs_subsample = probabilities[target_subsample.index]
        profit_sample = revenue(target_subsample, probs_subsample)
        values.append(profit_sample)
    values = pd.Series(values)  
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    count = 0
    for i in range(len(values)):
        if values[i] < 0:
            count += 1
    return print('\nСредняя прибыль', round(values.mean(),2),
                 '\n95% доверительный интервал от:',round(lower,2),'до:', round(upper,2),
                 '\nПроцент риска',round(count / len(values) * 100,3))

In [26]:
a = pd.Series([2, 2, 2, 3], index = [1, 1, 1, 2])
print(a[1])
print(a[a.index])
a = a.reset_index(drop = True)
print(a[1])
print(a[a.index])

1    2
1    2
1    2
dtype: int64
1    2
1    2
1    2
1    2
1    2
1    2
1    2
1    2
1    2
2    3
dtype: int64
2
0    2
1    2
2    2
3    3
dtype: int64


In [18]:
Bootstrap(target_valid_0, predictions_0)


Средняя прибыль 394284411.34 
95% доверительный интервал от: -69450566.85 до: 915496165.83 
Процент риска 6.1


In [19]:
Bootstrap(target_valid_1, predictions_1)


Средняя прибыль 454736359.1 
95% доверительный интервал от: 61071811.3 до: 855914149.14 
Процент риска 0.7


In [20]:
Bootstrap(target_valid_2, predictions_2)


Средняя прибыль 353664097.52 
95% доверительный интервал от: -162650946.94 до: 847761844.51 
Процент риска 7.6


### Вывод

`Регион 2`
- Самый низкий процент риска < 2,5%
- Отсутствует отрицательные значения доверительного интервала
- Наибольшая средняя прибыль

На основании вышеизложенного можно говорить о том, что `Регион 2` является наилучшим выбором из трех предложенных для разработки скважин

## Итоги

Были рассмотрены три региона с потенциально возможными местами для бурения новых скважин

Изначально, потенциально самым прибыльным регионом выглядел `Регион № 3`с самым большим показателем по среднему запасу 'Product'

После был рассчитан объём сырья для безубыточной разработки новой скважины (111.11)

Так как бюджет и количество потенциальных точек бурения ограничены, при помощи модели линейной регрессии был рассчитан средний запас предсказанного сырья и прибыли, которые для каждого из регионов равны:

`Регион 1`
- Средний запас сырья: 92.50
- Прибыль: 3 320,826 млн. руб.

`Регион 2`
- Средний запас сырья: 68.83 
- Прибыль: 2 415,087 млн. руб.

`Регион 3`
- Средний запас сырья: 95.00
- Прибыль: 2 539,916 млн. руб.

Средний запас сырья в каждом из трех регионов оказался недостаточен для безубыточной разработки скважин

На основании вышеизложенного была применена техника Bootstrap с 1000 выборок для определения распределения прибыли и расчета рисков, в ходе применения которой был сделан вывод, что Регион 2 является наилучшим выбором из трех предложенных для разработки, т.к. имеет самый низкий процент риска получения отрицательной прибыли и максимальную потенциально возможную прибыль из представленных для разработки регионов.